In [1]:
import warnings
import os
root_dir = os.path.abspath(".")
warnings.filterwarnings("ignore", message=".*force_all_finite.*", category=FutureWarning)  # Ignore some warnings

from dotenv import load_dotenv
import pandas as pd
load_dotenv()

# Custom packages
from utils.auth.hospital_profile import HospitalProfile
from models.embed.openai_embedding import PatientChartEmbedding
from models.data_container.plugin.patient_epurun import load
from pipeline.base_manager import MLPipeline
from pipeline.dfm_pipeline import DFMTransformer, DFMConfig, DynamicFactorModel

# Generate the hospital profile for epurun
print("[Run] Create hospital profile (epurun)")
hosp = HospitalProfile(hospital_id="epurun", hospital_name="이푸른병원")

print("[Run] Load data ...")
origin = load(root_dir)

print("[Run] Get embedding ...")
emb = PatientChartEmbedding(profile=hosp)
emb.initialize_qdrant(
    host=os.getenv("QDRANT_HOST"),
    port=os.getenv("QDRANT_PORT")
)
embed = emb.retrieve_embedding()

print("[Run] Run the pipeline")
model_config = DFMConfig(
    freq="W",  # Weekly prediction
    k_factors=5,  # Latent factors
    factor_order=1,  # AR
    error_cov_type="diagonal",
    forecast_steps=1,
    reduce_to=10
)
pipe = MLPipeline(
    profile=hosp,
    original=origin,
    embedding=embed.mergeable_embeddings(),
    transformer=DFMTransformer(),
    model=DynamicFactorModel(config=model_config),
    evaluator=None,
)
f = pipe.run("t1")

/Users/goonzard/Developer/ds-202501-health-econ-mlflow/.venv/lib/python3.12/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


[Run] Create hospital profile (epurun)
[Run] Load data ...
Loading ATC code map. Last Update 2025/09/12
[Run] Get embedding ...
2025-10-12 20:40:23 | INFO     | pipeline.PatientChartEmbedding | PatientChartEmbedding initialized [hospital_profile=이푸른병원 | collection_name=epurun_patient_embedding | vector_size=1536 | embedding_model=text-embedding-ada-002]
2025-10-12 20:40:23 | INFO     | pipeline.PatientChartEmbedding | Starting qdrant_initialization
2025-10-12 20:40:23 | INFO     | pipeline.PatientChartEmbedding | Parameters: qdrant_host=localhost, qdrant_port=6333, collection_name=epurun_patient_embedding, vector_size=1536
2025-10-12 20:40:23 | INFO     | pipeline.PatientChartEmbedding | Using existing collection: epurun_patient_embedding
2025-10-12 20:40:23 | INFO     | pipeline.PatientChartEmbedding | Parameters: collection_created=False, collection_exists=True
2025-10-12 20:40:23 | INFO     | pipeline.PatientChartEmbedding | Completed qdrant_initialization in 0.13s
2025-10-12 20:40:

Retrieving patient vectors: 100%|██████████| 11/11 [00:00<00:00, 32.26it/s]

2025-10-12 20:40:24 | INFO     | pipeline.PatientChartEmbedding | Completed qdrant_retrieval in 0.35s
2025-10-12 20:40:24 | INFO     | pipeline.PatientChartEmbedding | Metrics: qdrant_retrieval_duration_seconds=0.3545


[Run] Run the pipeline
2025-10-12 20:40:24 | INFO     | pipeline.DynamicFactorModel | DynamicFactorModel initialized [freq=W | k_factors=5 | factor_order=1 | error_cov_type=diagonal | forecast_steps=1 | reduce_to=10]


...

2025-10-12 20:41:38 | INFO     | pipeline.DynamicFactorModel | Fitting Dynamic Factor Model for substance famotidine from 2021-01-01 00:00:00 to 2023-01-01 00:00:00
2025-10-12 20:41:38 | INFO     | pipeline.DynamicFactorModel | Testing Dynamic Factor Model for substance famotidine from 2023-01-01 00:00:00 to 2023-01-29 00:00:00


/Users/goonzard/Developer/ds-202501-health-econ-mlflow/.venv/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


Fitting Dynamic Factor Model - startx
[[-17.349869    -0.9083103   10.975748   ...   3.1167073    3.9809835
    6.212879  ]
 [-17.294636    -0.91811776  11.043229   ...   3.1776018    4.0465364
    6.265774  ]
 [-17.616959    -1.0339873   10.7632065  ...   2.9705412    3.9974046
    6.230933  ]
 ...
 [-11.517849     0.9290774   15.324177   ...  12.427812     8.756157
   17.515097  ]
 [-11.712077     1.0972846   14.972408   ...  12.199347     8.783043
   17.642609  ]
 [-12.809968     1.3142955   14.951978   ...  11.502468     9.247616
   15.380294  ]]
shape (105, 10)
k_factors 5
factor_order 1


Fitting Dynamic Factor Models:  18%|█▊        | 43/241 [00:45<03:29,  1.06s/it]


KeyboardInterrupt: 